In [ ]:
!add-apt-repository -y ppa:jonathonf/ffmpeg-4 -q
!apt update -q
!apt install -y ffmpeg -q

In [ ]:

!pip install -q "datasets>=2.6.1"
!pip install -q "evaluate>=0.3.0"
!pip install -q git+https://github.com/huggingface/transformers.git@main
!pip install -q librosa
!pip install -q jiwer
!pip install -q gradio
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/evaluate.git@main



In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
model_name_or_path = "openai/whisper-small"
language ="English"
language_abbr ="en"
task = "transcribe"
dataset_name = "Trelis//llm-lingo"
org = "Trelis"
trained_adapter_name = "whisper-small-llm-lingo-adapters"
trained_model_name = "whisper-small-llm-lingo"
trained_adapter_repo = org + "/" + trained_adapter_name
trained_model_repo = org + '/' + trained_model_name


In [ ]:
from google.colab import files
uploaded = files.upload()
!pip install moviepy

from moviepy.editor import *

video = VideoFileClip("train1.mp4")
video.audio.write_audiofile("train1.mp3")

In [ ]:
from google.colab import files
uploaded = files.upload()
!pip install moviepy

from moviepy.editor import *

video = VideoFileClip("validation_MyVoice.mp4")
video.audio.write_audiofile("validation_MyVoice.mp3")

In [ ]:
!pip install -q transformers datasets librosa torchaudio

from transformers import pipeline
from transformers import (
    AutomaticSpeechRecognitionPipeline,
    WhisperTimeStampLogitsProcessor,
    WhisperForConditionalGeneration,
    WhisperTokenizer,
    WhisperProcessor,
)
import torch